# Uploading Custom Annotation Data to CAVE

In [1]:
from caveclient import CAVEclient
import numpy as np
import pandas as pd

In [4]:
annoDf = pd.read_csv('../../data/cell_bodies_sorted_by_neurite_path_with_location.csv')

In [5]:
annoDf.head()

,lores_id,hires_id_agglo,path_length_um,x,y,z
0,29371,15960395825,1568843.0,113536,28800,6392
1,18157,15960395825,1568843.0,114560,28544,6389
2,63875,15960395825,1568843.0,107328,34944,6508
3,99749,15960395825,1568843.0,100544,29184,6939
4,33793,15960395825,1568843.0,98816,36480,6710


In [6]:
annoDf.shape

(187052, 6)

# Create data for soma_ids_locations annotation table

In [7]:
df = annoDf.copy()

In [24]:
# upload random annotations
nrow = df.shape[0]
pt_position = df.apply(lambda row : np.array([row['x'], row['y'], row['z']]), axis=1)

data = pd.DataFrame.from_dict({
    'valid': np.repeat(True, nrow),
    'pt_position': pt_position,
    'tag': df.lores_id.astype(str).tolist(),
})

In [22]:
len(data.id.unique())

187052

In [13]:
data.shape

(187052, 4)

# CAVEclient

In [14]:
url = "https://global.brain-wire-test.org/"
datastack = "fish1_full"

In [15]:
client = CAVEclient(datastack_name=datastack, server_address=url)

In [16]:
client.info.get_datastacks()

['h01_c3_flat', 'fish1_full', 'fish1_test']

In [17]:
# get existing tables
all_tables = client.annotation.get_tables()
all_tables

['synapses_axax', 'synapses_axde', 'soma_ids_locations']

## Upload data

In [ ]:
import time
chunk_size = 10000  # Number of rows per chunk
i = 180000 # to end of the table

# for i in range(start, len(data), chunk_size):
chunk = data.iloc[i:i+chunk_size]
client.annotation.post_annotation_df(table_name='soma_ids_locations',
                                     df = chunk,
                                     position_columns=["pt_position"])
#     time.sleep(10)

## Get Data

In [45]:
client.annotation.get_annotation(table_name="soma_ids_locations",
                                 annotation_ids=[2])

[{'valid': True,
  'pt_position': [113536, 28800, 6392],
  'tag': '29371',
  'deleted': 'None',
  'id': 2,
  'superceded_id': None,
  'created': '2025-04-04 20:09:33.650068'}]

In [47]:
client.annotation.get_annotation(table_name="soma_ids_locations",
                                 annotation_ids=[187052])

[{'valid': True,
  'pt_position': [60032, 31104, 799],
  'tag': '43563',
  'created': '2025-04-04 20:17:37.953920',
  'superceded_id': None,
  'id': 187052,
  'deleted': 'None'}]

## Get Ready your Annotation Table to be Materialized

The new annotation table needs ingest_annotations workflow to be materialized. The workflow can be triggered in [Materialization Engine API doc](https://pcgv3local.brain-wire-test.org/materialize/api/doc). Run the ingest_annotations endpoint. 